In [1]:
#!python -m caffe2.python.operator_test.flip_op_test

In [2]:
import numpy as np
from caffe2.python import core, model_helper, workspace, utils
from game import InitGame, AddGamePlay
from modeling import AddConvModel
mini_batch=3

## Init first step

In [3]:
ZERO = np.zeros((mini_batch,1,19,19), dtype=np.float32)
ONE = np.ones((mini_batch,1,19,19), dtype=np.float32)
init_data = np.concatenate((ZERO,ZERO,ONE,ZERO,
                            ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,
                            ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,
                            ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,
                            ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,
                            ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,
                            ZERO,ZERO,ZERO,ZERO), axis=1)
workspace.FeedBlob("data", init_data)

model = model_helper.ModelHelper(name="model", arg_scope={"order": "NCHW"}, init_params=True)
AddConvModel(model, "data", dim_in=48)
AddGamePlay(model, "data", "predict")

workspace.RunNetOnce(model.param_init_net)
workspace.CreateNet(model.net, overwrite=True)
workspace.RunNet(model.net)

init_move = np.reshape(workspace.FetchBlob('predict')[0], (-1)) # shape=(361,)
top_choice = np.argsort(-init_move)[0:mini_batch] # the top K step

for i in range(mini_batch):
    x = top_choice[i]/19
    y = top_choice[i]%19
    init_data[i,1,x,y] = 1 # opponent plus (x,y)
    init_data[i,2,x,y] = 0 # empty minus (x,y)
    init_data[i,4,x,y] = 1 # last 1 step plus (x,y)
    init_data[i,12] = -1

workspace.FeedBlob("data", init_data)

Traceback for operator 45 in network model
game.py:69
<ipython-input-3-7edcc3e76aa0>:14
/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2882
/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2822
/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718
/usr/local/lib/python2.7/dist-packages/ipykernel/zmqshell.py:533
/usr/local/lib/python2.7/dist-packages/ipykernel/ipkernel.py:196
/usr/local/lib/python2.7/dist-packages/ipykernel/kernelbase.py:399
/usr/local/lib/python2.7/dist-packages/ipykernel/kernelbase.py:235
/usr/local/lib/python2.7/dist-packages/ipykernel/kernelbase.py:283
/usr/local/lib/python2.7/dist-packages/tornado/stack_context.py:277
/usr/local/lib/python2.7/dist-packages/zmq/eventloop/zmqstream.py:414
/usr/local/lib/python2.7/dist-packages/zmq/eventloop/zmqstream.py:472
/usr/local/lib/python2.7/dist-packages/zmq/eventloop/zmqstream.py:440
/usr/local/lib/python2.7/dist-packages/tornado/stack_context.py:277
/usr

RuntimeError: [enforce fail at concat_split_op.h:105] split_tensor.size() == OutputSize(). 12 vs 11 Error from operator: 
input: "data" input: "split_size" output: "layer0" output: "layer1" output: "layer2" output: "layer3" output: "layer4to9" output: "layer10" output: "layer11" output: "layer12to19" output: "layer20to35" output: "layer36to43" output: "layer44to47" name: "" type: "Split" arg { name: "axis" i: 1 }

In [ ]:
for i in range(20):
    workspace.RunNet(model.net)

In [ ]:
data=workspace.FetchBlob("data")

In [ ]:
data[0,11]